## Trabajo realizado por Benjamín Rubio y José Baboun
Importamos librerías necesarias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab
import networkx as nx
from collections import deque
import geopandas as gpd
import math
from shapely.geometry import Point
import itertools

## Funciones básicas

La mayoría de estas funciones son búsquedas tipo BFS modificadas según lo buscado

### get_size:
- Input: Grafo (G), la lista de nodos ya visitados (V_) y un nodo (u).
- Output: Tamano del ideal de u dentro de los nodos no visitados.

In [2]:
def get_size(G, V_, u):
    
    if V_[u]:
        return 0
    
    V = V_.copy()
    
    ans = 1;  V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                ans += 1; V[v] = 1
                Q.append(v)
    
    return ans

### visit:
- Input: Grafo (G), la lista de nodos ya visitados (V) y un nodo (u).
- Output: Marca como visitados todos los nodos dentro del ideal de u no visitados. Retorna la lista actualizada y la cantidad de nodos visitados.

In [3]:
def visit(G, V, u):
    
    if V[u]:
        return V, 0
    
    V[u] = 1; ans = 0
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft(); ans += 1
        for v in G.predecessors(u):
            if not V[v]:
                V[v] = 1
                Q.append(v)
    
    return V, ans

### get_size_weight:
- Input: Grafo (G), lista de pesos por nodo (W), la lista de nodos ya visitados (V_) y un nodo (u).
- Output: Tamaño del ideal de u dentro de los nodos no visitados y su peso.

In [4]:
def get_size_weight(G, W, V_, u):
    
    if V_[u]:
        return 0, 0
    
    V = V_.copy()
    
    s = 1; w = W[u]; V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                s += 1; w += W[v]; V[v] = 1
                Q.append(v)
    
    return s, w

### get_size_precalc:
- Input: Grafo (G), lista de pesos de ideal por nodo precalculados (W), la lista de nodos ya visitados (V_) y un nodo (u).
- Output: Tamaño del ideal de u dentro de los nodos no visitados acumulando los ideales con peso menor a 200.

In [5]:
def get_size_precalc(G, W, V_, u):
    
    if V_[u]:
        return 0
    
    if W[u] <= 200:
        return 1
    
    V = V_.copy()
    
    s = 1; V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                s += 1; V[v] = 1
                if W[v] > 200:
                    Q.append(v)
    
    return s

### add_visit:
- Input: Grafo (G), la lista de nodos ya visitados (V) y un nodo (u).
- Output: None. Suma 1 a todos los nodos del ideal de u no visitados en V.

In [6]:
def add_visit(G, V, u):
    
    V_ = [0 for u in V]
    
    V[u] += 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V_[v]:
                V[v] += 1; V_[v] = 1
                Q.append(v)

### get_ideal:
- Input: Grafo (G), la lista de nodos ya visitados (V_) y un nodo (u).
- Output: Lista del ideal de u con nodos no visitados.

In [7]:
def get_ideal(G, V_, u):
    
    if V_[u]:
        return list()
    
    V = V_.copy()
    s = [u];  V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                s.append(v)
                V[v] = 1; Q.append(v)
    
    return s

###  find_order:
- Input: Grafo (G) dirigido, la lista de nodos ya visitados (V), nodos a ordenar (sample) y nodo raíz (s).
- Output: Nodos en (sample) ordenados según orden en el que aparecen en el BFS.

In [8]:
def find_order(G, V, sample, s):
    ans = []; Q = deque([]); Q.append(s)
    while Q:
        u = Q.popleft()
        if u in sample:
            ans.append(u)
        for v in G.predecessors(u):
            if not V[v]:
                Q.append(v)
    return ans

## Funciones para algoritmo y búsqueda robusta

### get_size_robust:
- Input: Grafo (G), la lista de nodos ya visitados (V_) y un nodo (node).
- Output: Cantidad de nodos no alcanzables de la raíz en caso que no se pueda pasar por (node). Equivalente a tamaño de ideal "robusto".

In [9]:
def get_size_robust(G, V_, root, node):
    
    N = len(V_)
    
    if V_[node]:
        return 0
    
    if node == root:
        return N - sum(V_)
    
    V = [0] * N
    
    V[root] = 1; Q = deque([root]);
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v] and v != node:
                V[v] = 1
                Q.append(v)

    return sum([1 for u in range(N) if V_[u] + V[u] == 0])

### get_ideal_robust:
- Input: Grafo (G), la lista de nodos ya visitados (V_) y un nodo (node).
- Output: Nodos no alcanzables de la raíz en caso que no se pueda pasar por (node). Equivalente a ideal "robusto".

In [10]:
def get_ideal_robust(G, V_, root, node):
    
    if root == node:
        return get_ideal(G, V_, root)
    
    N = len(V_)
    V = [0] * N
    V[root] = 1
    Q = deque([root])
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v] and v != node:
                V[v] = 1
                Q.append(v)

    return [i for i in range(N) if (V_[i] == 0 and V[i] == 0)]

### deactivate_robust:
- Input: Grafo (G), la lista de nodos ya visitados (V_) y un nodo (node).
- Output: Lista de nodos visitados desactivando aquellos en el ideal robusto de (node).

In [11]:
def deactivate_robust(G, V_, root, node):
    N = len(V_)
    V = [0] * N
    V[root] = 1; Q = deque([root]);
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v] and v != node:
                V[v] = 1
                Q.append(v)

    return [0 if V_[u] + V[u] == 0 else 1 for u in range(N)]

### add_visit_robust:
- Input: Grafo (G), la lista de nodos ya visitados (V) y un nodo (node).
- Output: Lista de nodos visitados original luego de marcar como visitados los nodos en el ideal robusto de (node).

In [12]:
def add_visit_robust(G, V, root, node):
    N = len(V)
    V[node] = 1
    I = get_ideal_robust(G, V, node)
    return [1 if (i in I or V[i]) else 0 for i in range(N) ]

### get_size_weight_robust:
- Input: Grafo (G), lista de pesos en nodos (W), la lista de nodos ya visitados (V_) y un nodo (node).
- Output: Cantidad de nodos en el ideal robusto de (node) y suma de sus pesos en forma de tupla.

In [13]:
def get_size_weight_robust(G, W, V_, root, node):
    N = len(V_)
    
    if V_[node]:
        return 0 , 0
    
    if node == root:
        return N - sum(V_), sum(W) - sum([W[i] for i in range(N) if V_[i]])
    
    V = [0] * N
    
    V[root] = 1; Q = deque([root]);
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v] and v != node:
                V[v] = 1
                Q.append(v)
                
    I = [i for i in range(N) if (V_[i] == 0 and V[i] == 0)]
    return sum([1 for i in I]), sum([W[i] for i in I])

### get_size_precalc_robust:
- Input: Grafo (G), lista de pesos en ideales de nodos (W), la lista de nodos ya visitados (V_) y un nodo (node).
- Output: Cantidad de nodos en el ideal robusto de (node) agrupando nodos con pesos < 200.

In [14]:
def get_size_precalc_robust(G, W, V_, root, node):
    
    N = len(V_)
    
    if V_[node]:
        return 0
    
    if W[node] <= 200:
        return 1
    
    V = [0] * N
    
    s = 1; V[node] = 1
    Q = deque([]); Q.append(root)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v] and v != node:
                s += 1; V[v] = 1
                Q.append(v)
                
    return sum([1 for i in range(N) if (V_[i] == 0 and V[i] == 0 and W[i] > 200) ])